In [1]:
import os
import pandas as pd
import mne
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import argparse
import glob
import cmlreaders as cml
import json
from matplotlib.ticker import FuncFormatter
import warnings
import re
from scipy import signal
from scipy.stats import zscore

beh_events_df = pd.read_csv('James_new_data/EFRCourierOpenLoop (3)/EFRCourierOpenLoop/sub-R1724J/ses-0/beh/sub-R1724J_ses-0_task-EFRCourierOpenLoop_events.tsv', sep='\t')
ieeg_events_df = pd.read_csv('James_new_data/EFRCourierOpenLoop (3)/EFRCourierOpenLoop/sub-R1724J/ses-0/ieeg/sub-R1724J_ses-0_task-EFRCourierOpenLoop_events.tsv', sep='\t')
channel_info = pd.read_csv('James_new_data/EFRCourierOpenLoop (3)/EFRCourierOpenLoop/sub-R1724J/ses-0/ieeg/sub-R1724J_ses-0_task-EFRCourierOpenLoop_acq-bipolar_channels.tsv', sep='\t')

beh_events_df['data_from']='behavior'
ieeg_events_df['data_from']='ieeg'
df_combined = pd.concat([beh_events_df, ieeg_events_df], ignore_index=True)
df_combined['data_from']
df_combined_sorted = df_combined.sort_values(by="onset", ascending=True)
df_combined_sorted = df_combined_sorted.reset_index(drop=True)



In [2]:
df_combined_sorted

,onset,duration,trial_type,trial_num,practice,item,item_num,serial_position,stim_condition,player_position,...,system_configs,store_mappings,data_from,stim_tag,amplitude,burst_fraction,burst_slow_frequency,electrode_neg,electrode_pos,frequency
0,0.000000,0.000000,ieeg_sync,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,behavior,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.000000,0.000000,ieeg_start,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,ieeg,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.000000,0.000000,ieeg_sync,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,ieeg,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.000000,0.000000,ieeg_start,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,behavior,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7.604435,0.000000,task_connected,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,ieeg,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20523,3295.081843,0.000000,fr_rec_word,3.0,False,MILK,132.0,NaN,NaN,NaN,...,NaN,NaN,behavior,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20524,3296.898572,3000.000000,stimming,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,ieeg,8Hz,0.7,0.25,8.0,41.0,40.0,200.0
20525,3302.896915,3000.000000,stimming,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,ieeg,8Hz,0.7,0.25,8.0,41.0,40.0,200.0
20526,3306.564125,0.000000,fr_stop,3.0,False,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,behavior,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
# =============================================================================
# CONFIGURATION
# =============================================================================

# Initialize reader and load events
evs =df_combined_sorted

# Define expected event counts
expected_counts = {
    'session_start': 1,
    'trial_start': 5,
    'encoding_word_start': 60,
    'stimming': '89 or 91',
    'pointing_start ': 65,
    'pointing_stop': 65,
    'fr_start': 5,
    'fr_stop': 5,
    'encoding_word_start': 60,
  
}

# Verify event counts
event_verification = {}
for event_type, expected in expected_counts.items():
    actual = len(evs[evs['trial_type'] == event_type])
    event_verification[event_type] = {
        'actual': actual,
        'expected': expected,
        'status': 'PASS' if (isinstance(expected, int) and actual == expected) or 
                           (isinstance(expected, str) and actual in [89, 91]) else 'FAIL'
    }
    print(f"{event_type}: {actual} / {expected} - {event_verification[event_type]['status']}")

print("\n1.2 STIMULATION DELIVERY VALIDATION") 
print("-"*30)


session_start: 1 / 1 - PASS
trial_start: 5 / 5 - PASS
encoding_word_start: 56 / 60 - FAIL
stimming: 84 / 89 or 91 - FAIL
pointing_start : 0 / 65 - FAIL
pointing_stop: 69 / 65 - FAIL
fr_start: 5 / 5 - PASS
fr_stop: 5 / 5 - PASS

1.2 STIMULATION DELIVERY VALIDATION
------------------------------


In [3]:
Indexs=evs[evs['trial_type']=='stimming'].index
for Index in Indexs:
    if (evs.iloc[Index + 1]['onset'] - evs.iloc[Index]['onset']) <= 3:
        evs.at[Index + 1, 'burst_slow_frequency'] = evs.iloc[Index]['burst_slow_frequency']
list_evs=evs[evs['trial_type']=='encoding_word_start']
list_evs['burst_slow_frequency'].unique()
evs['trial_num'] = evs['trial_num'].fillna(-999)
trials = evs['trial_num'].unique() 
trials_new = [trial for trial in trials if trial != -999]

# Get indices where trials start (non-practice trials only)
#EVS_START = evs[(evs['trial_type'] == 'trial_start') & (evs['practice'] == 0)].index
EVS_START = evs[(evs['trial_type'] == 'trial_start') ].index
EVS_START = np.array(EVS_START)

# Assign trial numbers to each segment
for index_d in range(len(EVS_START)):
    start_idx = EVS_START[index_d]
    
    # For the last trial, go to the end of the dataframe
    if index_d == len(EVS_START)-1:
        end_idx = len(evs) - 1
    else:
        end_idx = EVS_START[index_d + 1] - 1  # End before the next trial starts
    
    # Use the trial number from the sorted trials list
    if index_d < len(EVS_START)+1:
        evs.loc[start_idx:end_idx, 'trial_num'] = trials_new[index_d]
        print(evs.loc[end_idx, 'trial_num'])

-1.0
0.0
1.0
2.0
3.0


In [4]:
# Add phase information to events

def add_phase(evs):
    new_evs = pd.DataFrame([])
    for ind, list_evs in evs.groupby('trial_num'): # 'trial' is the aquivalent of a delivery day
        if list_evs['trial_num'].unique() != -999:
            list_evs['phase'] = 'nan' 
        if not list_evs[list_evs['trial_type'] == 'trial_start'].index.empty and not list_evs[list_evs['trial_type'] == 'deliveries_stop'].index.empty:
            # find indexes of encoding start and end
            first_pointing_index = min(list_evs[list_evs['trial_type']=='trial_start'].index)
            navigation_end_index = min(list_evs[list_evs['trial_type']=='deliveries_stop'].index) - 1
            list_evs.loc[first_pointing_index:navigation_end_index]['phase'] = 'encoding'
            first_pointing_index = min(list_evs[list_evs['trial_type']=='trial_start'].index)
            
            # find indexes of retrieval start and end
            rec_start_index = list_evs[list_evs['trial_type']=='fr_start'].index[0]
            rec_end_index = list_evs[list_evs['trial_type']=='fr_stop'].index[0]
            list_evs.loc[rec_start_index:rec_end_index]['phase'] = 'retrieval'
        new_evs = pd.concat([new_evs, list_evs])
    return new_evs

evs_new = add_phase(evs)


/usr/global/miniconda/py310_23.1.0-1/envs/workshop/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/global/miniconda/py310_23.1.0-1/envs/workshop/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [5]:
parameters_df2=pd.DataFrame()
for i,delivery_day_df in evs_new.groupby('trial_num'):
        if len(delivery_day_df['trial_type'].unique())>=0:
            encoding_df=delivery_day_df[delivery_day_df['phase']=='encoding']
            encoding_stim_num = len(encoding_df[encoding_df['trial_type']=='stimming'])
          
            retrieval_df=delivery_day_df[delivery_day_df['phase']=='retrieval']
            retrieval_stim_num = len(retrieval_df[retrieval_df['trial_type']=='stimming'])
           # recall_rate = len(delivery_day_df[delivery_day_df['recalled']==1]) / (len(delivery_day_df[delivery_day_df['recalled']==0]) + len(delivery_day_df[delivery_day_df['recalled']==1]))
           # intrusion = len(delivery_day_df[(delivery_day_df['intrusion']==-1) | (delivery_day_df['intrusion']>0)]) 
            stim_df=delivery_day_df[delivery_day_df['trial_type']=='stimming']
            burst_freq=stim_df['burst_slow_frequency'].unique()
            retrieval_item_list = set(retrieval_df['item'].dropna().str.lower().unique())
            encode_stim_set = set(encoding_df['item'].dropna().str.lower().unique())
            Number_send=len(encode_stim_set)
            idx_lead1_pos = channel_info[channel_info['lead1'].isin(delivery_day_df['electrode_pos'])].index
            idx_lead1_neg = channel_info[channel_info['lead1'].isin(delivery_day_df['electrode_neg'])].index
            idx_lead2_pos = channel_info[channel_info['lead2'].isin(delivery_day_df['electrode_pos'])].index  
            idx_lead2_neg = channel_info[channel_info['lead2'].isin(delivery_day_df['electrode_neg'])].index
            sets = [set(idx_lead1_pos), set(idx_lead1_neg), set(idx_lead2_pos), set(idx_lead2_neg)]
            from collections import Counter

            # Flatten all sets and count occurrences
            all_numbers = []
            for s in sets:
                all_numbers.extend(s)

            # Count how many times each number appears
            counts = Counter(all_numbers)

            # Get numbers that appear exactly twice
            numbers_twice = [num for num, count in counts.items() if count == 2]
            stim_channel=[]
            correct_recall=[]
            if len(all_numbers)>0:
                stim_channel=channel_info['name'].iloc[numbers_twice]
                correct_recall = len(retrieval_item_list.intersection(encode_stim_set))
                stim_channel=channel_info['name'].iloc[numbers_twice].values[0]
            parameters_df2 = parameters_df2.append({'Burst frequency (Hz)':burst_freq,'DD':delivery_day_df['trial_num'].unique()[0],
                                                   'Number of encoding stim':encoding_stim_num,'Number of retrieval stim (EFR)':retrieval_stim_num, 
                                                    'Number of item send':Number_send,'recalled item':correct_recall,'STIM_channel':stim_channel 
                                                   },ignore_index=True)
    #         parameters_df2 = parameters_df2.append({'Burst frequency (Hz)':burst_freq,'DD':delivery_day_df['trial'].unique()[0],
    #                                                'Number of encoding stim':encoding_stim_num,'Number of retrieval stim (EFR)':retrieval_stim_num,
    #                                                'Number of FSR stim':FSR_stim_num,'Number of FFR stim':FFR_stim_num},ignore_index=True)
parameters_df2

,Burst frequency (Hz),DD,Number of encoding stim,Number of retrieval stim (EFR),Number of item send,recalled item,STIM_channel
0,[],-999.0,0.0,0.0,0.0,[],[]
1,[],-1.0,0.0,0.0,8.0,[],[]
2,[3.0],0.0,6.0,15.0,12.0,4,LB10_LB11
3,[8.0],1.0,6.0,15.0,12.0,3,LC8_LC9
4,[3.0],2.0,6.0,15.0,12.0,3,LB10_LB11
5,[8.0],3.0,6.0,15.0,12.0,8,LC8_LC9


In [6]:
result_8HZ_encoding= evs_new[(evs_new['phase'] == 'encoding') & (evs_new['burst_slow_frequency'] == 8) & (evs_new['trial_type'] == 'stimming')]
result_3HZ_encoding= evs_new[(evs_new['phase'] == 'encoding') & (evs_new['burst_slow_frequency'] == 3)& (evs_new['trial_type'] == 'stimming')]
result_8HZ_retrieval= evs_new[(evs_new['phase'] == 'retrieval') & (evs_new['burst_slow_frequency'] == 8)& (evs_new['trial_type'] == 'stimming')]
result_3HZ_retrieval= evs_new[(evs_new['phase'] == 'retrieval') & (evs_new['burst_slow_frequency'] == 3)& (evs_new['trial_type'] == 'stimming')]
result_none_encoding = evs[(evs['trial_type'] == 'encoding_word_start') & (evs['stim_condition'] == False)]
print(f"\nStimulation distribution by phase :")
print(f"Encoding - 3 Hz: {len(result_3HZ_encoding)}, 8 Hz: {len(result_8HZ_encoding)}, None: {len(result_none_encoding)}")
print(f"Retrieval - 3 Hz: {len(result_3HZ_retrieval)}, 8 Hz: {len(result_8HZ_retrieval)}")


Stimulation distribution by phase :
Encoding - 3 Hz: 12, 8 Hz: 12, None: 29
Retrieval - 3 Hz: 30, 8 Hz: 30


In [8]:
evs_new['trial_type'].unique()

array(['ieeg_sync', 'ieeg_start', 'task_connected', 'task_configured',
       'session_start', 'town_learning_start', 'town_learning_stop',
       'pointing_start', 'pointing_stop', 'player_position',
       'delivery_stop', 'trial_start', 'deliveries_start',
       'delivery_start', 'encoding_word_start', 'encoding_word_stop',
       'deliveries_stop', 'fixation_start', 'fixation_stop', 'high_beep',
       'fr_start', 'fr_rec_word', 'low_beep', 'fr_stop', 'stim_select',
       'stimming', 'efr_mark'], dtype=object)

In [7]:
import mne
raw = mne.io.read_raw_edf("James_new_data/EFRCourierOpenLoop (3)/EFRCourierOpenLoop/sub-R1724J/ses-0/ieeg/sub-R1724J_ses-0_task-EFRCourierOpenLoop_acq-bipolar_ieeg.edf", preload=False)

# Check the channel names
print(raw.ch_names)


Extracting EDF parameters from /home1/rcao92/FromJames/James_new_data/EFRCourierOpenLoop (3)/EFRCourierOpenLoop/sub-R1724J/ses-0/ieeg/sub-R1724J_ses-0_task-EFRCourierOpenLoop_acq-bipolar_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
['LP1-LP2', 'LP2-LP3', 'LP3-LP4', 'LP4-LP5', 'LP5-LP6', 'LP6-LP7', 'LP7-LP8', 'LA1-LA2', 'LA2-LA3', 'LA3-LA4', 'LA4-LA5', 'LA5-LA6', 'LA6-LA7', 'LA7-LA8', 'LA8-LA9', 'LA9-LA10', 'LA10-LA11', 'LA11-LA12', 'LB1-LB2', 'LB2-LB3', 'LB3-LB4', 'LB4-LB5', 'LB5-LB6', 'LB6-LB7', 'LB7-LB8', 'LB8-LB9', 'LB9-LB10', 'LB10-LB11', 'LB11-LB12', 'LC1-LC2', 'LC2-LC3', 'LC3-LC4', 'LC4-LC5', 'LC5-LC6', 'LC6-LC7', 'LC7-LC8', 'LC8-LC9', 'LC9-LC10', 'LC10-LC11', 'LC11-LC12', 'LD1-LD2', 'LD2-LD3', 'LD3-LD4', 'LD4-LD5', 'LD5-LD6', 'LD6-LD7', 'LD7-LD8', 'LD8-LD9', 'LD9-LD10', 'LD10-LD11', 'LD11-LD12', 'LL1-LL2', 'LL2-LL3', 'LL3-LL4', 'LL4-LL5', 'LL5-LL6', 'LL6-LL7', 'LL7-LL8', 'RA1-RA2', 'RA2-RA3', 'RA3-RA4', 'RA4-RA5', 'RA5-RA6', 'RA6

In [10]:
raw.ch_names[31]

'LC3-LC4'

In [11]:
event_new['electrode_neg'].unique()



NameError: name 'event_new' is not defined

In [ ]:
events_onset=event_new['onset']
events_type=event_new['burst_slow_frequency']

In [ ]:
FromJames/James_new_data/EFRCourierOpenLoop (3)/EFRCourierOpenLoop/sub-R1724J/ses-0/ieeg/sub-R1724J_ses-0_task-EFRCourierOpenLoop_acq-bipolar_ieeg.edf

In [8]:
 # sampling rate in Hz
raw = mne.io.read_raw_edf('James_new_data/EFRCourierOpenLoop (3)/EFRCourierOpenLoop/sub-R1724J/ses-0/ieeg/sub-R1724J_ses-0_task-EFRCourierOpenLoop_acq-bipolar_ieeg.edf', preload=False)
sr =  raw.info['sfreq']
def segment(result_df,sfreq,tmin,tmax):
    events_samples = result_df['onset']*sfreq   # convert ms → s → samples
    events = np.zeros((len(events_samples), 3), dtype=int)
    events[:, 0] = events_samples                  # sample numbers
    events[:, 1] = 0                               # previous value, 0 if no trigger channel
    #events[:, 2] = [event_id_dict[str(etype)] for etype in events_type]  # event IDs
    events[:, 2] = 1
   
    epochs = mne.Epochs(raw, events, tmin=tmin, tmax=tmax, baseline=(None, 0), preload=True)
    data = epochs.get_data()
    return data
eeg_8HZ = segment(result_8HZ_encoding,sr,-1,4)
eeg_3HZ = segment(result_3HZ_encoding,sr,-1,4)
eeg_8HZ_retrieval = segment(result_8HZ_retrieval,sr,-1,10)
eeg_3HZ_retrieval = segment(result_3HZ_retrieval,sr,-1,10)
eeg_no_encoding = segment(result_none_encoding,sr,-1,4)
total_samples = eeg_8HZ.shape[2]
time_axis = np.linspace(-1000, time_window, total_samples)

print(f"Sample rate: {sr} Hz")
print(f"EEG data shapes:")
print(f"  8 Hz encoding: {eeg_8HZ.shape}")
print(f"  3 Hz encoding: {eeg_3HZ.shape}")
print(f"  No stimuli encoding: {eeg_no_encoding.shape}")
print(f"Time axis range: {time_axis[0]:.1f} to {time_axis[-1]:.1f} ms")


Extracting EDF parameters from /home1/rcao92/FromJames/James_new_data/EFRCourierOpenLoop (3)/EFRCourierOpenLoop/sub-R1724J/ses-0/ieeg/sub-R1724J_ses-0_task-EFRCourierOpenLoop_acq-bipolar_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Not setting metadata
12 matching events found
Setting baseline interval to [-1.0, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 12 events and 5001 original time points ...
0 bad epochs dropped
Not setting metadata
12 matching events found
Setting baseline interval to [-1.0, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 12 events and 5001 original time points ...
0 bad epochs dropped
Not setting metadata
30 matching events found
Setting baseline interval to [-1.0, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 30 events and 11001 original time points ...
0 bad 

NameError: name 'time_window' is not defined

In [ ]:
result_8HZ_encoding

In [9]:
def identify_stimulated_channels(delivery_day_df, channel_info):
    idx_lead1_pos = channel_info[channel_info['lead1'].isin(delivery_day_df['electrode_pos'])].index
    idx_lead1_neg = channel_info[channel_info['lead1'].isin(delivery_day_df['electrode_neg'])].index
    idx_lead2_pos = channel_info[channel_info['lead2'].isin(delivery_day_df['electrode_pos'])].index  
    idx_lead2_neg = channel_info[channel_info['lead2'].isin(delivery_day_df['electrode_neg'])].index
    All_contact_included=[idx_lead1_pos,idx_lead1_neg,idx_lead2_pos,idx_lead2_neg]
    All_contact_included_unique= np.unique(All_contact_included)
    sets = [set(idx_lead1_pos), set(idx_lead1_neg), set(idx_lead2_pos), set(idx_lead2_neg)]
    from collections import Counter

    # Flatten all sets and count occurrences
    all_numbers = []
    for s in sets:
        all_numbers.extend(s)

    # Count how many times each number appears
    counts = Counter(all_numbers)

    # Get numbers that appear exactly twice
    numbers_twice = [num for num, count in counts.items() if count == 2]

    numbers_once = [num for num, count in counts.items() if count == 1]
    stim_channel=[]
    correct_recall=[]
    if len(all_numbers) > 0:
        stim_pairs = channel_info['name'].iloc[numbers_twice].values[0]
        all_the_rest = channel_info['group'] != channel_info['group'].iloc[numbers_twice].values[0]
        The_shank = channel_info['group'] == channel_info['group'].iloc[numbers_twice].values[0]
        print(f"Before: {The_shank.sum()}")

        # Set specific rows to False
        The_shank.iloc[All_contact_included_unique] = False
        index_other=numbers_twice
        channel_stimuli_indices=The_shank[The_shank].index
        index_other=all_the_rest[all_the_rest].index
        contact_stimuli_indices=numbers_twice
        print(f"After: {The_shank.sum()}")
    return index_other, channel_stimuli_indices, contact_stimuli_indices
Index_other_8, Channel_stimuli_8, Contact_stimuli_8 = identify_stimulated_channels(result_8HZ_encoding, channel_info)
Index_other_3, Channel_stimuli_3, Contact_stimuli_3 = identify_stimulated_channels(result_3HZ_encoding, channel_info)

Before: 11
After: 8
Before: 11
After: 8


In [ ]:
Index_other_8

In [10]:
print("\n2.1 EEG Response for simulation Visualization")
print("-"*30)
bed_channel=[]
# Create comprehensive visualization
fig = plt.figure(figsize=(20, 16))
channel = [i for i in range(np.shape(eeg_8HZ)[1]) if i not in bed_channel]

# Define stimulus period in time coordinates
stim_start_time, stim_end_time = 0, 3000  # ms

# Plot 1: Overall EEG responses (2x2 grid)
ax1 = plt.subplot(3, 3, 1)
avg_eeg = np.mean(eeg_8HZ[:, channel, :], 0)
plt.plot(time_axis, np.mean(avg_eeg[:, :], 0), linewidth=2, color='red')
plt.axvline(x=0, color='k', linestyle='-')
plt.axhline(y=0, color='k', linestyle='--')
plt.xlabel('Time (ms)')
plt.ylabel('Voltage')
plt.title('8 Hz Encoding (All Channels)')
plt.axvspan(stim_start_time, stim_end_time, alpha=0.3, color='lightgrey')

ax2 = plt.subplot(3, 3, 2)
avg_eeg = np.mean(eeg_3HZ[:, channel, :], 0)
plt.plot(time_axis, np.mean(avg_eeg[:, :], 0), linewidth=2, color='blue')
plt.axvline(x=0, color='k', linestyle='-')
plt.axhline(y=0, color='k', linestyle='--')
plt.xlabel('Time (ms)')
plt.ylabel('Voltage')
plt.title('3 Hz Encoding (All Channels)')
plt.axvspan(stim_start_time, stim_end_time, alpha=0.3, color='lightgrey')

ax3 = plt.subplot(3, 3, 3)
avg_eeg = np.mean(eeg_no_encoding[:, channel, :], 0)
plt.plot(time_axis, np.mean(avg_eeg[:, :], 0), linewidth=2, color='green')
plt.axvline(x=0, color='k', linestyle='-')
plt.axhline(y=0, color='k', linestyle='--')
plt.xlabel('Time (ms)')
plt.ylabel('Voltage')
plt.title('No Stimuli Encoding')
plt.axvspan(stim_start_time, stim_end_time, alpha=0.3, color='lightgrey')

# Plot 2: Channel-specific analysis for 8 Hz
if Contact_stimuli_8:
    avg_eeg_other_8 = np.mean(eeg_8HZ[:, Index_other_8, :], axis=1)
    avg_eeg_stimuli_8 = np.mean(eeg_8HZ[:, Contact_stimuli_8, :], axis=1)
    avg_eeg_channel_8 = np.mean(eeg_8HZ[:, Channel_stimuli_8, :], axis=1)

    ax4 = plt.subplot(3, 3, 4)
    plt.plot(time_axis, np.mean(avg_eeg_other_8, axis=0), linewidth=2, color='blue', label='Other Channels')
    plt.axhline(y=0, color='k', linestyle='--')
    plt.xlabel('Time (ms)')
    plt.ylabel('Voltage')
    plt.title('8 Hz: Other Channels')
    plt.axvspan(stim_start_time, stim_end_time, alpha=0.3, color='lightgrey')
    ax4_max = np.max(np.abs(avg_eeg_stimuli_8))/10
    ax4.set_ylim(-ax4_max, ax4_max)

    ax5 = plt.subplot(3, 3, 5)
    plt.plot(time_axis, np.mean(avg_eeg_stimuli_8, axis=0), linewidth=2, color='red', label='Stimulated Pairs')
    plt.axhline(y=0, color='k', linestyle='--')
    plt.xlabel('Time (ms)')
    plt.ylabel('Voltage')
    plt.title('8 Hz: Stimulated Pairs')
    plt.axvspan(stim_start_time, stim_end_time, alpha=0.3, color='lightgrey')
    ax5_max = np.max(np.abs(avg_eeg_stimuli_8))
    ax5.set_ylim(-ax5_max, ax5_max)

    ax6 = plt.subplot(3, 3, 6)
    plt.plot(time_axis, np.mean(avg_eeg_channel_8, axis=0), linewidth=2, color='orange', label='Stimulated Shank')
    plt.axhline(y=0, color='k', linestyle='--')
    plt.xlabel('Time (ms)')
    plt.ylabel('Voltage')
    plt.title('8 Hz: Stimulated Shank')
    plt.axvspan(stim_start_time, stim_end_time, alpha=0.3, color='lightgrey')
    ax6_max = np.max(np.abs(avg_eeg_stimuli_8))/10
    ax6.set_ylim(-ax6_max, ax6_max)

# Plot 3: Channel-specific analysis for 3 Hz
if Contact_stimuli_3:
    avg_eeg_other_3 = np.mean(eeg_3HZ[:, Index_other_3, :], axis=1)
    avg_eeg_stimuli_3 = np.mean(eeg_3HZ[:, Contact_stimuli_3, :], axis=1)
    avg_eeg_channel_3 = np.mean(eeg_3HZ[:, Channel_stimuli_3, :], axis=1)

    ax7 = plt.subplot(3, 3, 7)
    plt.plot(time_axis, np.mean(avg_eeg_other_3, axis=0), linewidth=2, color='blue', label='Other Channels')
    plt.axhline(y=0, color='k', linestyle='--')
    plt.xlabel('Time (ms)')
    plt.ylabel('Voltage')
    plt.title('3 Hz: Other Channels')
    plt.axvspan(stim_start_time, stim_end_time, alpha=0.3, color='lightgrey')

    ax8 = plt.subplot(3, 3, 8)
    mean_stimuli_3 = np.mean(avg_eeg_stimuli_3, axis=0)
    plt.plot(time_axis, mean_stimuli_3, linewidth=2, color='cyan', label='Stimulated Pairs')
    plt.axhline(y=0, color='k', linestyle='--')
    plt.xlabel('Time (ms)')
    plt.ylabel('Voltage')
    plt.title('3 Hz: Stimulated Pairs')
    plt.axvspan(stim_start_time, stim_end_time, alpha=0.3, color='lightgrey')

    # Set ax8 y-limits to its maximum value
    ax8_max = np.max(np.abs(mean_stimuli_3))
    ax8.set_ylim(-ax8_max, ax8_max)

    # Calculate reduced range for ax7 and ax9
    reduced_range = ax8_max / 10

    ax9 = plt.subplot(3, 3, 9)
    plt.plot(time_axis, np.mean(avg_eeg_channel_3, axis=0), linewidth=2, color='purple', label='Stimulated Shank')
    plt.axhline(y=0, color='k', linestyle='--')
    plt.xlabel('Time (ms)')
    plt.ylabel('Voltage')
    plt.title('3 Hz: Stimulated Shank')
    plt.axvspan(stim_start_time, stim_end_time, alpha=0.3, color='lightgrey')

    # Set ax7 and ax9 y-limits to ax8_max / 10
    ax7.set_ylim(-reduced_range, reduced_range)
    ax9.set_ylim(-reduced_range, reduced_range)

plt.tight_layout()
plt.show()

fig, ax10 = plt.subplots()
ax10.plot(time_axis, avg_eeg_stimuli_3.T, linewidth=0.5)
ax10.axvline(x=0, color='k', linestyle='--')
ax10.set_xlim(-1000, 1000)
ax10.set_xlabel('Time (ms)')
ax10.set_ylabel('Voltage')
ax10.set_title('3 Hz Stimulated Pairs - Individual Trials')
plt.show()

fig, ax10 = plt.subplots()
ax10.plot(time_axis, avg_eeg_stimuli_8.T, linewidth=0.5)
ax10.axvline(x=0, color='k', linestyle='--')
ax10.set_xlim(-1000, 1000)
ax10.set_xlabel('Time (ms)')
ax10.set_ylabel('Voltage')
ax10.set_title('8 Hz Stimulated Pairs - Individual Trials')
plt.show()


 


2.1 EEG Response for simulation Visualization
------------------------------


NameError: name 'time_axis' is not defined

In [ ]:
print("\n2.1 EEG Response for simulation Visualization")
print("-"*30)
bed_channel=[]
# Create comprehensive visualization
fig = plt.figure(figsize=(20, 16))
channel = [i for i in range(np.shape(eeg_8HZ)[1]) if i not in bed_channel]
Index_other_8, Channel_stimuli_8, Contact_stimuli_8 = identify_stimulated_channels(result_8HZ_retrieval, channel_info)
Index_other_3, Channel_stimuli_3, Contact_stimuli_3 = identify_stimulated_channels(result_3HZ_retrieval, channel_info)
# Define stimulus period in time coordinates
stim_start_time, stim_end_time = 0, 3000
stim_start_time_2, stim_end_time_2 = 6000, 9000# ms
total_samples = eeg_8HZ_retrieval.shape[2]
time_window=10000
time_axis = np.linspace(-1000, time_window, total_samples)
# Plot 1: Overall EEG responses (2x2 grid)
ax1 = plt.subplot(3, 3, 1)
avg_eeg = np.mean(eeg_8HZ_retrieval[:, channel, :], 0)
plt.plot(time_axis, np.mean(avg_eeg[:, :], 0), linewidth=2, color='red')
plt.axvline(x=0, color='k', linestyle='-')
plt.axhline(y=0, color='k', linestyle='--')
plt.xlabel('Time (ms)')
plt.ylabel('Voltage')
plt.title('8 Hz Encoding (All Channels)')
plt.axvspan(stim_start_time, stim_end_time, alpha=0.3, color='lightgrey')
plt.axvspan(stim_start_time_2, stim_end_time_2, alpha=0.3, color='lightgrey')A
ax2 = plt.subplot(3, 3, 2)
avg_eeg = np.mean(eeg_3HZ_retrieval[:, channel, :], 0)
plt.plot(time_axis, np.mean(avg_eeg[:, :], 0), linewidth=2, color='blue')
plt.axvline(x=0, color='k', linestyle='-')
plt.axhline(y=0, color='k', linestyle='--')
plt.xlabel('Time (ms)')
plt.ylabel('Voltage')
plt.title('3 Hz Encoding (All Channels)')
plt.axvspan(stim_start_time, stim_end_time, alpha=0.3, color='lightgrey')
plt.axvspan(stim_start_time_2, stim_end_time_2, alpha=0.3, color='lightgrey')
ax3 = plt.subplot(3, 3, 3)
avg_eeg = np.mean(eeg_3HZ_retrieval[:, channel, :], 0)
plt.plot(time_axis, np.mean(avg_eeg[:, :], 0), linewidth=2, color='green')
plt.axvline(x=0, color='k', linestyle='-')
plt.axhline(y=0, color='k', linestyle='--')
plt.xlabel('Time (ms)')
plt.ylabel('Voltage')
plt.title('No Stimuli Encoding')
plt.axvspan(stim_start_time, stim_end_time, alpha=0.3, color='lightgrey')
plt.axvspan(stim_start_time_2, stim_end_time_2, alpha=0.3, color='lightgrey')
# Plot 2: Channel-specific analysis for 8 Hz
if Contact_stimuli_8:
    avg_eeg_other_8 = np.mean(eeg_8HZ_retrieval[:, Index_other_8, :], axis=1)
    avg_eeg_stimuli_8 = np.mean(eeg_8HZ_retrieval[:, Contact_stimuli_8, :], axis=1)
    avg_eeg_channel_8 = np.mean(eeg_8HZ_retrieval[:, Channel_stimuli_8, :], axis=1)

    ax4 = plt.subplot(3, 3, 4)
    plt.plot(time_axis, np.mean(avg_eeg_other_8, axis=0), linewidth=2, color='blue', label='Other Channels')
    plt.axhline(y=0, color='k', linestyle='--')
    plt.xlabel('Time (ms)')
    plt.ylabel('Voltage')
    plt.title('8 Hz: Other Channels')
    plt.axvspan(stim_start_time, stim_end_time, alpha=0.3, color='lightgrey')
    plt.axvspan(stim_start_time_2, stim_end_time_2, alpha=0.3, color='lightgrey')
    ax4_max = np.max(np.abs(avg_eeg_stimuli_8))/10
    ax4.set_ylim(-ax4_max, ax4_max)

    ax5 = plt.subplot(3, 3, 5)
    plt.plot(time_axis, np.mean(avg_eeg_stimuli_8, axis=0), linewidth=2, color='red', label='Stimulated Pairs')
    plt.axhline(y=0, color='k', linestyle='--')
    plt.xlabel('Time (ms)')
    plt.ylabel('Voltage')
    plt.title('8 Hz: Stimulated Pairs')
    plt.axvspan(stim_start_time, stim_end_time, alpha=0.3, color='lightgrey')
    plt.axvspan(stim_start_time_2, stim_end_time_2, alpha=0.3, color='lightgrey')
    ax5_max = np.max(np.abs(avg_eeg_stimuli_8))
    ax5.set_ylim(-ax5_max, ax5_max)

    ax6 = plt.subplot(3, 3, 6)
    plt.plot(time_axis, np.mean(avg_eeg_channel_8, axis=0), linewidth=2, color='orange', label='Stimulated Shank')
    plt.axhline(y=0, color='k', linestyle='--')
    plt.xlabel('Time (ms)')
    plt.ylabel('Voltage')
    plt.title('8 Hz: Stimulated Shank')
    plt.axvspan(stim_start_time, stim_end_time, alpha=0.3, color='lightgrey')
    plt.axvspan(stim_start_time_2, stim_end_time_2, alpha=0.3, color='lightgrey')
    ax6_max = np.max(np.abs(avg_eeg_stimuli_8))/10
    ax6.set_ylim(-ax6_max, ax6_max)

# Plot 3: Channel-specific analysis for 3 Hz
if Contact_stimuli_3:
    avg_eeg_other_3 = np.mean(eeg_3HZ_retrieval[:, Index_other_3, :], axis=1)
    avg_eeg_stimuli_3 = np.mean(eeg_3HZ_retrieval[:, Contact_stimuli_3, :], axis=1)
    avg_eeg_channel_3 = np.mean(eeg_3HZ_retrieval[:, Channel_stimuli_3, :], axis=1)

    ax7 = plt.subplot(3, 3, 7)
    plt.plot(time_axis, np.mean(avg_eeg_other_3, axis=0), linewidth=2, color='blue', label='Other Channels')
    plt.axhline(y=0, color='k', linestyle='--')
    plt.xlabel('Time (ms)')
    plt.ylabel('Voltage')
    plt.title('3 Hz: Other Channels')
    plt.axvspan(stim_start_time, stim_end_time, alpha=0.3, color='lightgrey')
    plt.axvspan(stim_start_time_2, stim_end_time_2, alpha=0.3, color='lightgrey')

    ax8 = plt.subplot(3, 3, 8)
    mean_stimuli_3 = np.mean(avg_eeg_stimuli_3, axis=0)
    plt.plot(time_axis, mean_stimuli_3, linewidth=2, color='cyan', label='Stimulated Pairs')
    plt.axhline(y=0, color='k', linestyle='--')
    plt.xlabel('Time (ms)')
    plt.ylabel('Voltage')
    plt.title('3 Hz: Stimulated Pairs')
    plt.axvspan(stim_start_time, stim_end_time, alpha=0.3, color='lightgrey')
    plt.axvspan(stim_start_time_2, stim_end_time_2, alpha=0.3, color='lightgrey')

    # Set ax8 y-limits to its maximum value
    ax8_max = np.max(np.abs(mean_stimuli_3))
    ax8.set_ylim(-ax8_max, ax8_max)

    # Calculate reduced range for ax7 and ax9
    reduced_range = ax8_max / 10

    ax9 = plt.subplot(3, 3, 9)
    plt.plot(time_axis, np.mean(avg_eeg_channel_3, axis=0), linewidth=2, color='purple', label='Stimulated Shank')
    plt.axhline(y=0, color='k', linestyle='--')
    plt.xlabel('Time (ms)')
    plt.ylabel('Voltage')
    plt.title('3 Hz: Stimulated Shank')
    plt.axvspan(stim_start_time, stim_end_time, alpha=0.3, color='lightgrey')
    plt.axvspan(stim_start_time_2, stim_end_time_2, alpha=0.3, color='lightgrey')

    # Set ax7 and ax9 y-limits to ax8_max / 10
    ax7.set_ylim(-reduced_range, reduced_range)
    ax9.set_ylim(-reduced_range, reduced_range)

plt.tight_layout()
plt.show()

fig, ax10 = plt.subplots()
ax10.plot(time_axis, avg_eeg_stimuli_3.T, linewidth=0.5)
ax10.axvline(x=0, color='k', linestyle='--')
ax10.set_xlim(-1000, 1000)
ax10.set_xlabel('Time (ms)')
ax10.set_ylabel('Voltage')
ax10.set_title('3 Hz Stimulated Pairs - Individual Trials')
plt.show()

fig, ax10 = plt.subplots()
ax10.plot(time_axis, avg_eeg_stimuli_8.T, linewidth=0.5)
ax10.axvline(x=0, color='k', linestyle='--')
ax10.set_xlim(-1000, 1000)
ax10.set_xlabel('Time (ms)')
ax10.set_ylabel('Voltage')
ax10.set_title('8 Hz Stimulated Pairs - Individual Trials')
plt.show()

In [ ]:
time_window

In [ ]:
true_indices = The_shank[The_shank].index


In [ ]:
# =============================================================================
# PHASE 3: CHANNEL QUALITY ASSESSMENT
# =============================================================================
print("\n" + "="*80)
print("PHASE 3: CHANNEL QUALITY ASSESSMENT")
print("-"*50)
channel = [i for i in range(np.shape(eeg_8HZ)[1]) if i not in bed_channel]
channels =channel_info['name']
print("-"*30)
  # Raw DATA :
  # Create subplots with 1 row and 2 columns

eeg_mean = np.mean(eeg_no_encoding[:, channel, :], 0)
# First subplot - Line plot
# Plot each channel (rows 1-9 from the mean result)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
for i in range(eeg_mean.shape[0]):
    ax1.plot(time_axis, eeg_mean[i, :], alpha=0.7)
ax1.set_xlabel('Time (ms)')
ax1.set_ylabel('Mean EEG Amplitude')
ax1.set_title('Mean EEG Signal Across Trials')
ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax1.grid(True, alpha=0.3)

# Second subplot - Heatmap
im = ax2.imshow(eeg_mean, aspect='auto', cmap='RdBu_r', origin='lower', 
                extent=[time_axis[0], time_axis[-1], 0, eeg_mean.shape[0]])
plt.colorbar(im, ax=ax2, label='Mean EEG Amplitude')
ax2.set_xlabel('Time (ms)')
ax2.set_ylabel('Channel')
ax2.set_title('Mean EEG Signal - Heatmap View')
# ax2.set_yticks(range(eeg_mean.shape[0]), [f'Ch {i+2}' for i in range(eeg_mean.shape[0])])

plt.tight_layout()
plt.show()

# Compute trial variability 
trial_variability = np.std(eeg_no_encoding[:,channel, :],axis=0)  # Shape: (channels, timepoints)
timepoint_variability = np.mean(trial_variability, axis=0)  # Average across channels
channel_variability = np.mean(trial_variability, axis=1)  # Average across timepoints

# Z-score channel variability to identify outliers
channel_variability_z = zscore(channel_variability)
problematic_channels = np.where(np.abs(channel_variability_z) > 2)[0]

print(f"Total channels analyzed: {len(channels)}")
print(f"Channels with high variability (|z-score| > 2): {len(problematic_channels)}")

if len(problematic_channels) > 0:
    print("\nPotential problematic channels:")
    for ch_idx in problematic_channels:
        if ch_idx < len(channels):
            print(f"  Channel {ch_idx}: {channels.iloc[ch_idx]} (z-score: {channel_variability_z[ch_idx]:.2f})")

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Channel Quality Assessment - Trials Variance Detection', fontsize=16)
# Heatmap of trial variability
ax1 = axes[0, 0]
im = ax1.imshow(trial_variability, aspect='auto', cmap='viridis', origin='lower',
                extent=[time_axis[0], time_axis[-1], 0, trial_variability.shape[0]])
ax1.set_xlabel('Time (ms)')
ax1.set_ylabel('Channels')
ax1.set_title('Trial Variability Heatmap')
plt.colorbar(im, ax=ax1, label='Standard Deviation')

# Average variability across time
ax2 = axes[0, 1]
ax2.plot(time_axis, timepoint_variability, linewidth=2)
ax2.set_xlabel('Time (ms)')
ax2.set_ylabel('Average Standard Deviation')
ax2.set_title('Average Trial Variability Across Time')
ax2.grid(True, alpha=0.3)
ax2.axvspan(stim_start_time, stim_end_time, alpha=0.3, color='lightgrey', label='Stimulus Period')
ax2.legend()

# Channel variability z-scores
ax3 = axes[1, 0]
ax3.plot(channel_variability_z, linewidth=2)
ax3.axhline(y=2, color='r', linestyle='--', label='Z-score = ±2')
ax3.axhline(y=-2, color='r', linestyle='--')
ax3.set_xlabel('Channel Number')
ax3.set_ylabel('Z-scored Standard Deviation')
ax3.set_title('Channel Variability Z-scores')
ax3.grid(True, alpha=0.3)
ax3.legend()
# Problematic channels over time
ax4 = axes[1, 1]
if len(problematic_channels) > 0:
    for ch in problematic_channels[:8]:  # Plot up to 8 problematic channels
        if ch < trial_variability.shape[0]:
            ax4.plot(time_axis, np.mean(eeg_no_encoding[:, ch, :], axis=0), 
                    label=f'Ch {ch}: {channels[ch] if ch < len(channels) else "Unknown"}', 
                    linewidth=2)
    ax4.set_xlabel('Time (ms)')
    ax4.set_ylabel('Standard Deviation')
    ax4.set_title('Mean over trials for Problematic Channels Detected by Trial Variability')
    ax4.grid(True, alpha=0.3)
    ax4.legend(fontsize=8)
    ax4.axvspan(stim_start_time, stim_end_time, alpha=0.3, color='lightgrey')

plt.tight_layout()
plt.show()

timepoint_variance = np.var(eeg_no_encoding[:,channel,:], axis=2)  # Shape: (trials, channels) - variance across timepoints
trial_averaged_variance = np.mean(timepoint_variance, axis=0)  # Shape: (channels,) - average across trials

# For visualization: compute variance for each channel at each timepoint (averaged across trials)
channel_timepoint_variance = np.var(np.mean(eeg_no_encoding[:,channel,:], axis=0), axis=1)  # Shape: (channels,) - variance across timepoints for trial-averaged data
timepoint_channel_variance = np.var(np.mean(eeg_no_encoding[:,channel,:], axis=0), axis=0)  # Shape: (timepoints,) - variance across channels for trial-averaged data

# Z-score channel variance to identify outliers
channel_variance_z = zscore(trial_averaged_variance)
problematic_channels = np.where(np.abs(channel_variance_z) > 2)[0]

print(f"Total channels analyzed: {len(channel)}")
print(f"Channels with high timepoint variance (|z-score| > 2): {len(problematic_channels)}")
if len(problematic_channels) > 0:
    print("\nPotnetial Problematic Channels:")
    for ch_idx in problematic_channels:
        if ch_idx < len(channels):
            print(f"  Channel {ch_idx}: {channels.iloc[ch_idx]} (z-score: {channel_variance_z[ch_idx]:.2f})")

# Visualization of artifact detection
fig, axes = plt.subplots(1, 3, figsize=(16, 12))
fig.suptitle('Channel Quality Assessment - Timepoint Variance Detection', fontsize=16)

# Heatmap of timepoint variance across trials and channels
ax1 = axes[0]
im = ax1.imshow(timepoint_variance.T, aspect='auto', cmap='viridis', origin='lower')
ax1.set_xlabel('Trials')
ax1.set_ylabel('Channels')
ax1.set_title('Timepoint Variance Heatmap (Trials × Channels)')
plt.colorbar(im, ax=ax1, label='Variance Across Timepoints')

# Channel variance z-scores
ax2 = axes[1]
ax2.plot(channel_variance_z, linewidth=2)
ax2.axhline(y=2, color='r', linestyle='--', label='Z-score = ±2')
ax2.axhline(y=-2, color='r', linestyle='--')
ax2.set_xlabel('Channel Number')
ax2.set_ylabel('Z-scored Timepoint Variance with trials')
ax2.set_title('Channel Timepoint Variance Z-scores')
ax2.grid(True, alpha=0.3)
ax2.legend()
trial_averaged_data=np.mean(eeg_no_encoding[:, :, :], axis=0)
# Problematic channels - timepoint variance profiles
ax4 = axes[2]
if len(problematic_channels) > 0:
    for ch in problematic_channels[:8]:  # Plot up to 8 problematic channels
        if ch < trial_averaged_data.shape[0]:
            channel_variance_over_time = np.mean(eeg_no_encoding[:, ch, :], axis=0)  # Variance across trials at each timepoint
            ax4.plot(time_axis, channel_variance_over_time, 
                    label=f'Ch {ch}: {channels.iloc[ch] if ch < len(channels) else "Unknown"}', 
                    linewidth=2)
    ax4.set_xlabel('Time (ms)')
    ax4.set_ylabel('Variance Across Trials')
    ax4.set_title('Variance Over Time for Problematic Channels')
    ax4.grid(True, alpha=0.3)
    ax4.legend(fontsize=8)
    ax4.axvspan(stim_start_time, stim_end_time, alpha=0.3, color='lightgrey')
else:
    ax4.text(0.5, 0.5, 'No problematic channels detected', 
             ha='center', va='center', transform=ax4.transAxes, fontsize=12)
    ax4.set_title('Mean over trial for Problematic Channels Detected by Temporal Variability within Trial')

plt.tight_layout()
plt.show()

In [ ]:
print(f"Total channels analyzed: {len(channels)}")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.fft import fft, fftfreq

mean_data = np.mean(eeg_no_encoding, axis=0)  # Shape: (53, 22000)

# FFT parameters
sampling_rate = 1000  # Hz - adjust this to your actual sampling rate
n_samples = mean_data.shape[1]

# Calculate frequency axis
freqs = fftfreq(n_samples, 1/sampling_rate)[:n_samples//2]  # Only positive frequencies

# Use fewer columns for larger subplots
n_channels = mean_data.shape[0]
n_cols = min(6, int(np.ceil(np.sqrt(n_channels))))
n_rows = int(np.ceil(n_channels / n_cols))

# Create larger figure
fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 25))
axes = axes.flatten()

# Plot FFT power for each channel
for channel in range(n_channels):
    # Compute FFT
    fft_data = fft(mean_data[channel, :])
    
    # Calculate power spectrum (magnitude squared)
    power_spectrum = np.abs(fft_data[:n_samples//2])**2
    
    # Plot without log transformation
    axes[channel].plot(freqs, power_spectrum, linewidth=0.8)
    axes[channel].set_title(channels.iloc[channel], fontsize=11)
    axes[channel].set_xlabel('Frequency (Hz)', fontsize=9)
    axes[channel].set_ylabel('Power', fontsize=9)
    axes[channel].tick_params(labelsize=8)
    axes[channel].grid(True, alpha=0.3)
    
    # Optional: limit frequency range to typical EEG bands (0-100 Hz)
    axes[channel].set_xlim(0, 100)

# Hide unused subplots
for i in range(n_channels, len(axes)):
    axes[i].set_visible(False)

# Optimize layout
plt.tight_layout(pad=1.5)
plt.savefig('line_noise.pdf', format='pdf', dpi=300, bbox_inches='tight')
plt.show()